In the first step, we are installing the necessary libraries used in this notebook


In [1]:
import numpy as np 
!pip install geopy
!pip install folium
import geopy
import pandas as pd 
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json 
from geopy.geocoders import Nominatim 

import requests 
from pandas.io.json import json_normalize 

import matplotlib.cm as cm
import matplotlib.colors as colors

from sklearn.cluster import KMeans

import folium 

print('Libraries imported.')


Libraries imported.


Inorder to scrape the Wikipedia website, we are going to use Beautifulsoup python package.Before doing that, we should install the beautifulsoup & html parsers packages into this notebook

In [2]:
!pip install beautifulsoup4
!pip install lxml
!pip install html5lib
!pip install requests


In the below lines of code,With the help of BeautifulSoup ,the toronto city neighborhood table is extracted and stored in a csv file called 'Wiki_scrape.csv'

In [3]:
from bs4 import BeautifulSoup
import csv

source = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').text
soup= BeautifulSoup(source,'lxml')

csv_file= open('Wiki_scrape.csv','w')

csv_writer= csv.writer(csv_file)
csv_writer.writerow(['Postcode','Borough','Neighborhood'])

tables = soup.find_all('table', class_='wikitable sortable')

for table in tables:
    ths = table.find_all('th')
    headings = [th.text.strip() for th in ths]
    if headings[:3] == ['Postcode','Borough','Neighborhood']:
        break

with open('soup', 'w') as toronto_df:
    for tr in table.find_all('tr'):
        tds = tr.find_all('td')
        if not tds:
            continue
        Postcode,Borough,Neighborhood = [td.text.strip() for td in tds[:3]]
        csv_writer.writerow([Postcode,Borough,Neighborhood])
        

Now using pandas library, the CSV file is read and stored in a pandas dataframe 'df'.But data wrangling is required for this dataframe inorder to proceed further.

In [4]:
df=pd.read_csv('Wiki_scrape.csv')
df.head()

,Postcode,Borough,Neighborhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront


In the below cell, we are dropping the rows where the value of Borough column is 'Not assigned'

In [5]:
df=df[df.Borough !='Not assigned']
df.head()

,Postcode,Borough,Neighborhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront
5,M6A,North York,Lawrence Heights
6,M6A,North York,Lawrence Manor


Now we are going to change'Not assigned' values in Neighborhood column with it's respective Borough.

In [6]:
df=df.astype('str')
df=df.replace('Not assigned','Queen\'s Park')
df.head()

,Postcode,Borough,Neighborhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront
5,M6A,North York,Lawrence Heights
6,M6A,North York,Lawrence Manor


As instructed, we are grouping the dataset based on each Borough's Postal code. 

In [7]:
df = df.groupby('Postcode').agg({'Borough':'first','Neighborhood':', '.join}).reset_index()
df

,Postcode,Borough,Neighborhood
0,M1B,Scarborough,"Rouge, Malvern"
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae
5,M1J,Scarborough,Scarborough Village
6,M1K,Scarborough,"East Birchmount Park, Ionview, Kennedy Park"
7,M1L,Scarborough,"Clairlea, Golden Mile, Oakridge"
8,M1M,Scarborough,"Cliffcrest, Cliffside, Scarborough Village West"
9,M1N,Scarborough,"Birch Cliff, Cliffside West"


In [8]:
df.rename(columns={"Postcode": "Postal Code"})


,Postal Code,Borough,Neighborhood
0,M1B,Scarborough,"Rouge, Malvern"
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae
5,M1J,Scarborough,Scarborough Village
6,M1K,Scarborough,"East Birchmount Park, Ionview, Kennedy Park"
7,M1L,Scarborough,"Clairlea, Golden Mile, Oakridge"
8,M1M,Scarborough,"Cliffcrest, Cliffside, Scarborough Village West"
9,M1N,Scarborough,"Birch Cliff, Cliffside West"


In [9]:
URL='https://cocl.us/Geospatial_data'
geo_df=pd.read_csv(URL)
geo_df=geo_df.astype('str')
geo_df

,Postal Code,Latitude,Longitude
0,M1B,43.806686299999996,-79.19435340000001
1,M1C,43.7845351,-79.16049709999999
2,M1E,43.7635726,-79.1887115
3,M1G,43.7709921,-79.21691740000001
4,M1H,43.773136,-79.23947609999999
5,M1J,43.7447342,-79.23947609999999
6,M1K,43.7279292,-79.26202940000002
7,M1L,43.711111700000004,-79.2845772
8,M1M,43.716316,-79.23947609999999
9,M1N,43.692657000000004,-79.2648481


In [10]:

import types
import pandas as pd
from botocore.client import Config
import ibm_boto3

def __iter__(self): return 0

client_51d2b1e122544064967f33b0e77e7d7e = ibm_boto3.client(service_name='s3',
    ibm_api_key_id='R0KTjAnpgukwCMcePdQjOmIkB72SP4SC-pSNSUcjHsHB',
    ibm_auth_endpoint="https://iam.ng.bluemix.net/oidc/token",
    config=Config(signature_version='oauth'),
    endpoint_url='https://s3-api.us-geo.objectstorage.service.networklayer.com')

body = client_51d2b1e122544064967f33b0e77e7d7e.get_object(Bucket='datascience-donotdelete-pr-eeqnw0g4imnqxo',Key='Toronto_df.csv')['Body']

if not hasattr(body, "__iter__"): body.__iter__ = types.MethodType( __iter__, body )

Toronto_df = pd.read_csv(body)
Toronto_df=Toronto_df.drop(Toronto_df.index[100])
Toronto_df


,Unnamed: 0,Postcode,Borough,Neighborhood,Latitude,Longitude
0,0.0,M1B,Scarborough,"Rouge, Malvern",43.806686,-79.194353
1,1.0,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union",43.784535,-79.160497
2,2.0,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,3.0,M1G,Scarborough,Woburn,43.770992,-79.216917
4,4.0,M1H,Scarborough,Cedarbrae,43.773136,-79.239476
5,5.0,M1J,Scarborough,Scarborough Village,43.744734,-79.239476
6,6.0,M1K,Scarborough,"East Birchmount Park, Ionview, Kennedy Park",43.727929,-79.262029
7,7.0,M1L,Scarborough,"Clairlea, Golden Mile, Oakridge",43.711112,-79.284577
8,8.0,M1M,Scarborough,"Cliffcrest, Cliffside, Scarborough Village West",43.716316,-79.239476
9,9.0,M1N,Scarborough,"Birch Cliff, Cliffside West",43.692657,-79.264848


In [11]:
print('The dataframe has {} boroughs and {} neighborhoods.'.format(
        len(Toronto_df['Borough'].unique()),
        Toronto_df.shape[0]
    )
)

The dataframe has 11 boroughs and 100 neighborhoods.


In [12]:
address = 'Toronto,Ontario'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto City are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto City are 43.653963, -79.387207.


In [13]:
map_Toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

for lat, lng, borough, neighborhood in zip(Toronto_df['Latitude'], Toronto_df['Longitude'], Toronto_df['Borough'], Toronto_df['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_Toronto)  
    
map_Toronto

In [70]:
North_York_data = Toronto_df[Toronto_df['Borough'] == 'North York'].reset_index(drop=True)
North_York_data.head()

,Unnamed: 0,Postcode,Borough,Neighborhood,Latitude,Longitude
0,17.0,M2H,North York,Hillcrest Village,43.803762,-79.363452
1,18.0,M2J,North York,"Fairview, Henry Farm, Oriole",43.778517,-79.346556
2,19.0,M2K,North York,Bayview Village,43.786947,-79.385975
3,20.0,M2L,North York,"Silver Hills, York Mills",43.757490,-79.374714
4,21.0,M2M,North York,"Newtonbrook, Willowdale",43.789053,-79.408493


In [71]:
map_North_York = folium.Map(location=[latitude, longitude], zoom_start=11)

for lat, lng, label in zip(North_York_data['Latitude'], North_York_data['Longitude'], North_York_data['Neighborhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_North_York)  
    
map_North_York

In [72]:
CLIENT_ID = 'B03XWJB3BDNQ2IUHER52X50QMTEZMLLQQUDLUL4EGTYXKT0Y' 
CLIENT_SECRET = 'MRK02P42E1LANXOEEZPXSQ3XMEXUPKQC3SJYMAZVCQU1AGP0' 
VERSION = '20180605'

In [73]:
North_York_data.loc[0,'Neighborhood']

'Hillcrest Village'

In [74]:
neighborhood_latitude = North_York_data.loc[0, 'Latitude'] 
neighborhood_longitude = North_York_data.loc[0, 'Longitude'] 

neighborhood_name = North_York_data.loc[0, 'Neighborhood'] 

print('Latitude and longitude values of {} are {}, {}.'.format(neighborhood_name, 
                                                               neighborhood_latitude, 
                                                               neighborhood_longitude))

Latitude and longitude values of Hillcrest Village are 43.8037622, -79.3634517.


In [75]:
LIMIT = 100
radius = 500
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    neighborhood_latitude, 
    neighborhood_longitude, 
    radius, 
    LIMIT)

In [76]:
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5e1aa5df83525f001b195f0b'},
 'response': {'headerLocation': 'Toronto',
  'headerFullLocation': 'Toronto',
  'headerLocationGranularity': 'city',
  'totalResults': 4,
  'suggestedBounds': {'ne': {'lat': 43.808262204500004,
    'lng': -79.3572281853783},
   'sw': {'lat': 43.7992621955, 'lng': -79.3696752146217}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '4ad9dce6f964a520651b21e3',
       'name': "Eagle's Nest Golf Club",
       'location': {'address': '10000 Dufferin Rd',
        'lat': 43.805454826002794,
        'lng': -79.36418592243415,
        'labeledLatLngs': [{'label': 'display',
          'lat': 43.805454826002794,
          'lng': -79.36418592243415}],
        'distance': 197,
        'cc': 'CA',
        'city': 'Toronto

In [31]:
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [32]:
venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()

,name,categories,lat,lng
0,Eagle's Nest Golf Club,Golf Course,43.805455,-79.364186
1,AY Jackson Pool,Pool,43.804515,-79.366138
2,Villa Madina,Mediterranean Restaurant,43.801685,-79.363938
3,Duncan Creek Park,Dog Run,43.805539,-79.360695


In [33]:
print('{} venues were returned by Foursquare.'.format(nearby_venues.shape[0]))

4 venues were returned by Foursquare.


In [77]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        results = requests.get(url).json()["response"]['groups'][0]['items']
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [78]:
North_York_venues = getNearbyVenues(names=North_York_data['Neighborhood'],
                                   latitudes=North_York_data['Latitude'],
                                   longitudes=North_York_data['Longitude']
                                  )

Hillcrest Village
Fairview, Henry Farm, Oriole
Bayview Village
Silver Hills, York Mills
Newtonbrook, Willowdale
Willowdale South
York Mills West
Willowdale West
Parkwoods
Don Mills North
Flemingdon Park, Don Mills South
Bathurst Manor, Downsview North, Wilson Heights
Northwood Park, York University
CFB Toronto, Downsview East
Downsview West
Downsview Central
Downsview Northwest
Victoria Village
Bedford Park, Lawrence Manor East
Lawrence Heights, Lawrence Manor
Glencairn
Downsview, North Park, Upwood Park
Humber Summit
Emery, Humberlea


In [79]:
print(North_York_venues.shape)
North_York_venues.head()

(240, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Hillcrest Village,43.803762,-79.363452,Eagle's Nest Golf Club,43.805455,-79.364186,Golf Course
1,Hillcrest Village,43.803762,-79.363452,AY Jackson Pool,43.804515,-79.366138,Pool
2,Hillcrest Village,43.803762,-79.363452,Villa Madina,43.801685,-79.363938,Mediterranean Restaurant
3,Hillcrest Village,43.803762,-79.363452,Duncan Creek Park,43.805539,-79.360695,Dog Run
4,"Fairview, Henry Farm, Oriole",43.778517,-79.346556,The LEGO Store,43.778207,-79.343483,Toy / Game Store


In [80]:
North_York_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
"Bathurst Manor, Downsview North, Wilson Heights",19,19,19,19,19,19
Bayview Village,4,4,4,4,4,4
"Bedford Park, Lawrence Manor East",25,25,25,25,25,25
"CFB Toronto, Downsview East",2,2,2,2,2,2
Don Mills North,4,4,4,4,4,4
Downsview Central,3,3,3,3,3,3
Downsview Northwest,5,5,5,5,5,5
Downsview West,6,6,6,6,6,6
"Downsview, North Park, Upwood Park",4,4,4,4,4,4


In [81]:
print('There are {} uniques categories.'.format(len(North_York_venues['Venue Category'].unique())))

There are 104 uniques categories.


In [82]:

North_York_onehot = pd.get_dummies(North_York_venues[['Venue Category']], prefix="", prefix_sep="")

North_York_onehot['Neighborhood'] = North_York_venues['Neighborhood'] 

fixed_columns = [North_York_onehot.columns[-1]] + list(North_York_onehot.columns[:-1])
North_York_onehot = North_York_onehot[fixed_columns]

North_York_onehot.head()

,Neighborhood,Accessories Store,Airport,American Restaurant,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Bakery,Bank,Bar,Baseball Field,Basketball Court,Beer Store,Bike Shop,Boutique,Bridal Shop,Bubble Tea Shop,Burger Joint,Burrito Place,Bus Station,Bus Stop,Business Service,Butcher,Cafeteria,Café,Candy Store,Caribbean Restaurant,Chinese Restaurant,Clothing Store,Coffee Shop,Comfort Food Restaurant,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop,Deli / Bodega,Department Store,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Electronics Store,Empanada Restaurant,Falafel Restaurant,Fast Food Restaurant,Fireworks Store,Food & Drink Shop,Food Court,Food Truck,Fried Chicken Joint,Frozen Yogurt Shop,Furniture / Home Store,Gas Station,Gift Shop,Golf Course,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Hockey Arena,Home Service,Hotel,Ice Cream Shop,Indian Restaurant,Indonesian Restaurant,Italian Restaurant,Japanese Restaurant,Jewelry Store,Juice Bar,Liquor Store,Lounge,Luggage Store,Massage Studio,Mediterranean Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Movie Theater,Park,Pet Store,Pharmacy,Pizza Place,Plaza,Pool,Portuguese Restaurant,Pub,Ramen Restaurant,Restaurant,Salon / Barbershop,Sandwich Place,Shopping Mall,Smoothie Shop,Sporting Goods Shop,Steakhouse,Supermarket,Supplement Shop,Sushi Restaurant,Tea Room,Thai Restaurant,Theater,Toy / Game Store,Video Game Store,Video Store,Vietnamese Restaurant,Wings Joint,Women's Store
0,Hillcrest Village,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,Hillcrest Village,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,Hillcrest Village,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,Hillcrest Village,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,"Fairview, Henry Farm, Oriole",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0


In [83]:
North_York_grouped = North_York_onehot.groupby('Neighborhood').mean().reset_index()
North_York_grouped

,Neighborhood,Accessories Store,Airport,American Restaurant,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Bakery,Bank,Bar,Baseball Field,Basketball Court,Beer Store,Bike Shop,Boutique,Bridal Shop,Bubble Tea Shop,Burger Joint,Burrito Place,Bus Station,Bus Stop,Business Service,Butcher,Cafeteria,Café,Candy Store,Caribbean Restaurant,Chinese Restaurant,Clothing Store,Coffee Shop,Comfort Food Restaurant,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop,Deli / Bodega,Department Store,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Electronics Store,Empanada Restaurant,Falafel Restaurant,Fast Food Restaurant,Fireworks Store,Food & Drink Shop,Food Court,Food Truck,Fried Chicken Joint,Frozen Yogurt Shop,Furniture / Home Store,Gas Station,Gift Shop,Golf Course,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Hockey Arena,Home Service,Hotel,Ice Cream Shop,Indian Restaurant,Indonesian Restaurant,Italian Restaurant,Japanese Restaurant,Jewelry Store,Juice Bar,Liquor Store,Lounge,Luggage Store,Massage Studio,Mediterranean Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Movie Theater,Park,Pet Store,Pharmacy,Pizza Place,Plaza,Pool,Portuguese Restaurant,Pub,Ramen Restaurant,Restaurant,Salon / Barbershop,Sandwich Place,Shopping Mall,Smoothie Shop,Sporting Goods Shop,Steakhouse,Supermarket,Supplement Shop,Sushi Restaurant,Tea Room,Thai Restaurant,Theater,Toy / Game Store,Video Game Store,Video Store,Vietnamese Restaurant,Wings Joint,Women's Store
0,"Bathurst Manor, Downsview North, Wilson Heights",0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.052632,0.0,0.000000,0.00,0.000000,0.000000,0.000000,0.052632,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.0,0.000000,0.000000,0.00,0.000000,0.000000,0.105263,0.00,0.000000,0.00,0.000000,0.000000,0.052632,0.000000,0.000000,0.052632,0.000000,0.00,0.000000,0.00,0.0,0.052632,0.00,0.00,0.000000,0.000000,0.052632,0.052632,0.000000,0.052632,0.000000,0.00,0.00,0.000000,0.000000,0.00,0.00,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.00,0.052632,0.000000,0.000000,0.000000,0.000000,0.052632,0.052632,0.000000,0.00,0.00,0.00,0.000000,0.052632,0.000000,0.052632,0.052632,0.000000,0.000000,0.000000,0.052632,0.000000,0.052632,0.000000,0.00,0.000000,0.000000,0.000000,0.052632,0.000000,0.000000,0.000000
1,Bayview Village,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.250000,0.0,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.0,0.250000,0.000000,0.00,0.250000,0.000000,0.000000,0.00,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.00,0.0,0.000000,0.00,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.00,0.000000,0.000000,0.00,0.00,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.250000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.00,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2,"Bedford Park, Lawrence Manor East",0.000000,0.0,0.040000,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.040000,0.0,0.040000,0.000000,0.00,0.000000,0.000000,0.080000,0.04,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.00,0.0,0.080000,0.00,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.04,0.040000,0.000000,0.00,0.00,0.000000,0.000000,0.000000,0.04,0.000000,0.080000,0.000000,0.000000,0.040000,0.040000,0.000000,0.000000,0.0,0.00,0.000000,0.000000,0.000000,0.000000,0.040000,0.040000,0.080000,0.000000,0.00,0.00,0.04,0.000000,0.040000,0.000000,0.080000,0.000000,0.000000,0.000000,0.000000,0.0000

In [84]:
num_top_venues = 5

for hood in North_York_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = North_York_grouped[North_York_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Bathurst Manor, Downsview North, Wilson Heights----
                       venue  freq
0                Coffee Shop  0.11
1                Supermarket  0.05
2  Middle Eastern Restaurant  0.05
3                   Pharmacy  0.05
4                      Diner  0.05


----Bayview Village----
                 venue  freq
0   Chinese Restaurant  0.25
1                 Café  0.25
2                 Bank  0.25
3  Japanese Restaurant  0.25
4         Liquor Store  0.00


----Bedford Park, Lawrence Manor East----
                  venue  freq
0           Coffee Shop  0.08
1        Sandwich Place  0.08
2    Italian Restaurant  0.08
3  Fast Food Restaurant  0.08
4           Pizza Place  0.08


----CFB Toronto, Downsview East----
                venue  freq
0             Airport   0.5
1                Park   0.5
2   Accessories Store   0.0
3       Movie Theater   0.0
4  Miscellaneous Shop   0.0


----Don Mills North----
                  venue  freq
0  Caribbean Restaurant  0.25
1  Gym / Fitness C

In [85]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [86]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = North_York_grouped['Neighborhood']

for ind in np.arange(North_York_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(North_York_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,"Bathurst Manor, Downsview North, Wilson Heights",Coffee Shop,Gas Station,Bank,Bridal Shop,Deli / Bodega,Restaurant,Sandwich Place,Shopping Mall,Diner,Pizza Place
1,Bayview Village,Japanese Restaurant,Café,Bank,Chinese Restaurant,Women's Store,Coffee Shop,Comfort Food Restaurant,Concert Hall,Construction & Landscaping,Convenience Store
2,"Bedford Park, Lawrence Manor East",Fast Food Restaurant,Sandwich Place,Italian Restaurant,Coffee Shop,Pizza Place,Café,Butcher,Indian Restaurant,Pet Store,Pharmacy
3,"CFB Toronto, Downsview East",Airport,Park,Women's Store,Diner,Clothing Store,Coffee Shop,Comfort Food Restaurant,Concert Hall,Construction & Landscaping,Convenience Store
4,Don Mills North,Caribbean Restaurant,Café,Gym / Fitness Center,Japanese Restaurant,Diner,Coffee Shop,Comfort Food Restaurant,Concert Hall,Construction & Landscaping,Convenience Store


In [87]:

kclusters = 5

North_York_grouped_clustering = North_York_grouped.drop('Neighborhood', 1)

kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(North_York_grouped_clustering)

kmeans.labels_[0:10] 

array([2, 0, 2, 0, 0, 3, 2, 2, 0, 4], dtype=int32)

In [88]:

neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

North_York_merged = North_York_data

North_York_merged = North_York_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')



,Unnamed: 0,Postcode,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,17.0,M2H,North York,Hillcrest Village,43.803762,-79.363452,0.0,Golf Course,Mediterranean Restaurant,Pool,Dog Run,Women's Store,Department Store,Clothing Store,Coffee Shop,Comfort Food Restaurant,Concert Hall
1,18.0,M2J,North York,"Fairview, Henry Farm, Oriole",43.778517,-79.346556,2.0,Clothing Store,Fast Food Restaurant,Coffee Shop,Japanese Restaurant,Food Court,Toy / Game Store,Electronics Store,Bus Station,Women's Store,Jewelry Store
2,19.0,M2K,North York,Bayview Village,43.786947,-79.385975,0.0,Japanese Restaurant,Café,Bank,Chinese Restaurant,Women's Store,Coffee Shop,Comfort Food Restaurant,Concert Hall,Construction & Landscaping,Convenience Store
3,20.0,M2L,North York,"Silver Hills, York Mills",43.757490,-79.374714,1.0,Cafeteria,Women's Store,Chinese Restaurant,Coffee Shop,Comfort Food Restaurant,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop,Deli / Bodega
4,21.0,M2M,North York,"Newtonbrook, Willowdale",43.789053,-79.408493,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [134]:
North_York_merged['Cluster Labels']=North_York_merged['Cluster Labels'].fillna(0)
North_York_merged['Cluster Labels']=North_York_merged['Cluster Labels'].astype(int)
North_York_merged['Cluster Labels']

0     0
1     2
2     0
3     1
4     0
5     2
6     0
7     2
8     0
9     0
10    2
11    2
12    2
13    0
14    2
15    3
16    2
17    2
18    2
19    2
20    0
21    0
22    3
23    4
Name: Cluster Labels, dtype: int64

In [135]:
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

markers_colors = []
for lat, lon, poi, cluster in zip(North_York_merged['Latitude'], North_York_merged['Longitude'], North_York_merged['Neighborhood'], North_York_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

In [136]:
North_York_merged.loc[North_York_merged['Cluster Labels'] == 0, North_York_merged.columns[[1] + list(range(5, North_York_merged.shape[1]))]]

,Postcode,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M2H,-79.363452,0,Golf Course,Mediterranean Restaurant,Pool,Dog Run,Women's Store,Department Store,Clothing Store,Coffee Shop,Comfort Food Restaurant,Concert Hall
2,M2K,-79.385975,0,Japanese Restaurant,Café,Bank,Chinese Restaurant,Women's Store,Coffee Shop,Comfort Food Restaurant,Concert Hall,Construction & Landscaping,Convenience Store
4,M2M,-79.408493,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,M2P,-79.400049,0,Convenience Store,Bank,Park,Women's Store,Diner,Clothing Store,Coffee Shop,Comfort Food Restaurant,Concert Hall,Construction & Landscaping
8,M3A,-79.329656,0,Food & Drink Shop,Fireworks Store,Bus Stop,Park,Women's Store,Department Store,Clothing Store,Coffee Shop,Comfort Food Restaurant,Concert Hall
9,M3B,-79.352188,0,Caribbean Restaurant,Café,Gym / Fitness Center,Japanese Restaurant,Diner,Coffee Shop,Comfort Food Restaurant,Concert Hall,Construction & Landscaping,Convenience Store
13,M3K,-79.464763,0,Airport,Park,Women's Store,Diner,Clothing Store,Coffee Shop,Comfort Food Restaurant,Concert Hall,Construction & Landscaping,Convenience Store
20,M6B,-79.445073,0,Park,Asian Restaurant,Pub,Japanese Restaurant,Department Store,Chinese Restaurant,Clothing Store,Coffee Shop,Comfort Food Restaurant,Concert Hall
21,M6L,-79.490074,0,Construction & Landscaping,Bakery,Basketball Court,Park,Women's Store,Diner,Clothing Store,Coffee Shop,Comfort Food Restaurant,Concert Hall


In [137]:
North_York_merged.loc[North_York_merged['Cluster Labels'] == 1, North_York_merged.columns[[1] + list(range(5, North_York_merged.shape[1]))]]

,Postcode,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
3,M2L,-79.374714,1,Cafeteria,Women's Store,Chinese Restaurant,Coffee Shop,Comfort Food Restaurant,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop,Deli / Bodega


In [138]:
North_York_merged.loc[North_York_merged['Cluster Labels'] == 2, North_York_merged.columns[[1] + list(range(5, North_York_merged.shape[1]))]]

,Postcode,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
1,M2J,-79.346556,2,Clothing Store,Fast Food Restaurant,Coffee Shop,Japanese Restaurant,Food Court,Toy / Game Store,Electronics Store,Bus Station,Women's Store,Jewelry Store
5,M2N,-79.408493,2,Sushi Restaurant,Ramen Restaurant,Coffee Shop,Sandwich Place,Pizza Place,Café,Hotel,Middle Eastern Restaurant,Movie Theater,Lounge
7,M2R,-79.442259,2,Coffee Shop,Pharmacy,Pizza Place,Grocery Store,Butcher,Home Service,Discount Store,Women's Store,Department Store,Comfort Food Restaurant
10,M3C,-79.340923,2,Clothing Store,Coffee Shop,Asian Restaurant,Gym,Beer Store,Sporting Goods Shop,Concert Hall,Japanese Restaurant,Italian Restaurant,Dim Sum Restaurant
11,M3H,-79.442259,2,Coffee Shop,Gas Station,Bank,Bridal Shop,Deli / Bodega,Restaurant,Sandwich Place,Shopping Mall,Diner,Pizza Place
12,M3J,-79.487262,2,Caribbean Restaurant,Coffee Shop,Massage Studio,Falafel Restaurant,Bar,Diner,Comfort Food Restaurant,Concert Hall,Construction & Landscaping,Convenience Store
14,M3L,-79.506944,2,Grocery Store,Park,Bank,Hotel,Shopping Mall,Diner,Clothing Store,Coffee Shop,Comfort Food Restaurant,Concert Hall
16,M3N,-79.520999,2,Grocery Store,Athletics & Sports,Gym / Fitness Center,Liquor Store,Discount Store,Women's Store,Dim Sum Restaurant,Coffee Shop,Comfort Food Restaurant,Concert Hall
17,M4A,-79.315572,2,Coffee Shop,Grocery Store,Portuguese Restaurant,Hockey Arena,Women's Store,Dim Sum Restaurant,Clothing Store,Comfort Food Restaurant,Concert Hall,Construction & Landscaping
18,M5M,-79.419750,2,Fast Food Restaurant,Sandwich Place,Italian Restaurant,Coffee Shop,Pizza Place,Café,Butcher,Indian Restaurant,Pet Store,Pharmacy


In [139]:
North_York_merged.loc[North_York_merged['Cluster Labels'] == 3, North_York_merged.columns[[1] + list(range(5, North_York_merged.shape[1]))]]

,Postcode,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
15,M3M,-79.495697,3,Food Truck,Business Service,Baseball Field,Women's Store,Diner,Coffee Shop,Comfort Food Restaurant,Concert Hall,Construction & Landscaping,Convenience Store
22,M9L,-79.565963,3,Food Truck,Pizza Place,Home Service,Empanada Restaurant,Women's Store,Dim Sum Restaurant,Clothing Store,Coffee Shop,Comfort Food Restaurant,Concert Hall


In [140]:
North_York_merged.loc[North_York_merged['Cluster Labels'] == 4, North_York_merged.columns[[1] + list(range(5, North_York_merged.shape[1]))]]

,Postcode,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
23,M9M,-79.532242,4,Baseball Field,Women's Store,Discount Store,Coffee Shop,Comfort Food Restaurant,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop,Deli / Bodega


End of Notebook